In [23]:
import nomic
import json
nomic.login("nk-2LSEn5FB5h04Dn0pFLuZ9jDw7ZaMbPpYPV88wbKKASc")
#- Scientific programming
import numpy as np
import pandas as pd
from tqdm import tqdm
import os


from nomic import AtlasDataset

from unidecode import unidecode
import re
import datetime

dataset = AtlasDataset('hivemind/tweets-from-members-of-us-congress-from-all-time-updated-2024-10-08')

df = dataset.maps[0].data.df

congress_name_list = list(np.unique(list(df['name'])))
congress_handles = [list(df['twitter_lower'][df['name'] == cn])[0] for cn in congress_name_list]


df_congress = pd.DataFrame({'twitter_lower': congress_handles, 'congress_name_list': congress_name_list})
df_congress = df_congress.merge(df.drop_duplicates(subset='twitter_lower'), on='twitter_lower', how='left') ## ignore the column <115><2017><senate><179> and <118><2024><house><142>

2024-10-21 12:00:40.920 | INFO     | nomic.dataset:__init__:768 - Loading existing dataset `hivemind/tweets-from-members-of-us-congress-from-all-time-updated-2024-10-08`.
2024-10-21 12:00:42.999 | WARNING  | nomic.data_operations:df:923 - Converting to pandas dataframe. This may materialize a large amount of data into memory.
2024-10-21 12:00:43.001 | INFO     | nomic.data_operations:_download_data:902 - Downloading data
100%|██████████| 129/129 [00:00<00:00, 5092.95it/s]
2024-10-21 12:00:43.847 | INFO     | nomic.data_operations:_load_data:872 - Loading data
100%|██████████| 129/129 [00:01<00:00, 66.73it/s] 


In [24]:
df_congress.head()

,twitter_lower,congress_name_list,tweetId,postedAt,name,state,party,chamber,years,source,text,id
0,repfinkenauer,Abby Finkenauer D-IA,1266367737145683968,2020-05-29 13:56:11.167,Abby Finkenauer D-IA,IA,D,House,2019_2020,PolitWoops,Minor league baseball is a vital community ins...,2010694
1,repspanberger,Abigail Spanberger D-VA,1377973148675665920,2021-04-02 13:16:14.804,Abigail Spanberger D-VA,VA,D,House,2021_2022,PolitWoops,The #AmericanRescuePlan will provide funds to ...,2511843
2,repkinzinger,Adam Kinzinger R-IL,837318315202719744,2017-03-02 15:06:57.623,Adam Kinzinger R-IL,IL,R,House,2017_2018,PolitWoops,"Thanks, @Joepa84 - I currently serve in the 11...",1141249
3,repadamschiff,Adam Schiff D-CA,493881370440392704,2014-07-28 22:11:09.242,Adam Schiff D-CA,CA,D,House,2013_2014,PolitWoops,Concerned about #IdentityTheft? I'm hosting a ...,376089
4,repadamsmith,Adam Smith D-WA,1511117188941459456,2022-04-04 23:03:07.700,Adam Smith D-WA,WA,D,House,2021_2022,PolitWoops,"Tonight the House is voting on H.R. 1916, the ...",2753835


In [89]:
import os

file_list = os.listdir('/cis/home/tchen94/my_documents/congressional-twin/experiments/tianyi/Bill Votes/Education')
file_name = file_list[3]

read_file_path = '/cis/home/tchen94/my_documents/congressional-twin/experiments/tianyi/Bill Votes/Education/' + file_name

bill_name = file_name.split("congress_votes_")[1].split(".csv")[0]
bill_name

'118-2024_h427'

In [90]:
df_vote_result=pd.read_csv(read_file_path, header=1)
df_vote_result.head()

,person,state,district,vote,name,party
0,400004,AL,4,Yea,Rep. Robert Aderholt [R],Republican
1,400030,GA,2,Nay,Rep. Sanford Bishop [D],Democrat
2,400033,OR,3,Nay,Rep. Earl Blumenauer [D],Democrat
3,400052,TX,26,Yea,Rep. Michael Burgess [R],Republican
4,400057,CA,41,Yea,Rep. Ken Calvert [R],Republican


In [91]:
rep_or_senate=df_vote_result['name'][0].split(' [')[0].split('. ')[0]
print(rep_or_senate)

Rep


In [92]:
df_vote_result['matched-name'] = ''
df_vote_result['twitter-handle'] = ''
df_vote_result['matched-state'] = ''


for i, row in df_vote_result.iterrows():
    name = unidecode(row['name'].split(' [')[0].split(rep_or_senate+'. ')[1])
    party_val = row['party'][0]
    state_val = row['state']
    candidate_names = df_congress.loc[(df_congress['party'] == party_val) & (df_congress['state'] == state_val), 'congress_name_list'].apply(unidecode)
    name_parts = name.split(" ")

    for napa in name_parts:
        if candidate_names.str.contains(napa).any():
            index = candidate_names[candidate_names.str.contains(napa)].index[0]


            df_vote_result.at[i, 'matched-name'] = df_congress.loc[index]['congress_name_list']
            df_vote_result.at[i, 'twitter-handle'] = df_congress.loc[index]['twitter_lower']
            df_vote_result.at[i, 'matched-party']=df_congress.loc[index]['party']
            df_vote_result.at[i, 'matched-state']=df_congress.loc[index]['state']
        else:
            df_vote_result.at[i, 'matched-name'] = np.nan
            df_vote_result.at[i, 'twitter-handle'] = np.nan
            df_vote_result.at[i, 'matched-party']=np.nan
            df_vote_result.at[i, 'matched-state']=np.nan

df_vote_result.head()



/tmp/ipykernel_3912803/524609642.py:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if candidate_names.str.contains(napa).any():
/tmp/ipykernel_3912803/524609642.py:15: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  index = candidate_names[candidate_names.str.contains(napa)].index[0]


,person,state,district,vote,name,party,matched-name,twitter-handle,matched-state,matched-party
0,400004,AL,4,Yea,Rep. Robert Aderholt [R],Republican,"Aderholt, Robert",robert_aderholt,AL,R
1,400030,GA,2,Nay,Rep. Sanford Bishop [D],Democrat,"Bishop Jr., Sanford",sanfordbishop,GA,D
2,400033,OR,3,Nay,Rep. Earl Blumenauer [D],Democrat,"Blumenauer, Earl",repblumenauer,OR,D
3,400052,TX,26,Yea,Rep. Michael Burgess [R],Republican,"Burgess, Michael",michaelcburgess,TX,R
4,400057,CA,41,Yea,Rep. Ken Calvert [R],Republican,"Calvert, Ken",kencalvert,CA,R


## Warning: there will be unmatched CPs:

In [93]:
na_rows = df_vote_result[df_vote_result['matched-name'].isna()]
print(na_rows)


     person state  district vote                              name  \
7    400075    SC         6  Nay      Rep. James “Jim” Clyburn [D]   
308  456820    IL        15  Yea              Rep. Mary Miller [R]   
320  456835    NM         3  Nay   Rep. Teresa Leger Fernandez [D]   
387  456914    NC        11  Yea  Rep. Charles (Chuck) Edwards [R]   
418  456946    TX        38  Yea              Rep. Wesley Hunt [R]   
425  456955    RI         1  Nay                 Rep. Gabe Amo [D]   
426  456956    UT         2  Yea            Rep. Celeste Maloy [R]   
427  456957    NY        26  Nay          Rep. Timothy Kennedy [D]   
428  456958    CA        20  Yea               Rep. Vince Fong [R]   
429  456959    OH         6  Yea            Rep. Michael Rulli [R]   
430  456960    CO         4  Yea               Rep. Greg Lopez [R]   

          party matched-name twitter-handle matched-state matched-party  
7      Democrat          NaN            NaN           NaN           NaN  
308  Republ

In [95]:

save_file_path = '/cis/home/tchen94/my_documents/congressional-twin/experiments/tianyi/Matched Bill Votes/Education'

df_vote_result.to_csv(os.path.join(save_file_path, bill_name + '.csv'), index=False)
